# Numba

In [1]:
from src.Sudoku import Sudoku
import numpy as np


grid1 = """000801000
    000000043
    500000000
    000070800
    000000100
    020030000
    600000075
    003400000
    000200600"""

sudoku = Sudoku(grid1)


In [2]:
from numba import jit, prange

def calculate_candidates(sudoku):

    candidates = np.zeros([9, 9]) 

    for y in range(9):
        for x in range(9):
            if sudoku.get_cell(x, y) != 0:
                candidates[y][x] = 0
                continue

            row = sudoku.get_row(y)
            col = sudoku.get_col(x)
            sq_index = x // 3 + 3 * (y // 3)
            sq = sudoku.get_square(sq_index)
            invalid_numbers = np.concatenate((row, col, sq.flatten()))
            valid_numbers = np.setdiff1d(np.arange(1, 10), invalid_numbers)
            candidates[y][x] = len(valid_numbers)

    return candidates


@jit(nopython=True)
def custom_setdiff1d(ar1, ar2):
    result = []
    
    ar2_set = set(ar2)
    
    for item in ar1:
        if item not in ar2_set:
            result.append(item)
    
    return np.array(result)

@jit(nopython=True, parallel=True)
def calculate_candidates_2(grid: np.array):

    candidates = np.zeros((9, 9, 9), dtype=np.bool_)

    for y in prange(9):
        for x in prange(9):
            if grid[y, x] != 0:
                continue

            row = grid[y, :]
            col = grid[:, x]
            y0 = (y // 3) * 3
            x0 = (x // 3) * 3
            sq = grid[y0 : y0 + 3, x0 : x0 + 3]
            invalid_numbers = np.concatenate((row, col, sq.flatten()))
            valid_numbers = custom_setdiff1d(np.arange(1, 10), invalid_numbers)
            candidates[y, x, valid_numbers - 1] = 1

    return candidates


In [8]:
import time

start_time = time.time()

for _ in range(100000):
    c = calculate_candidates_2(sudoku.grid)

end_time = time.time()

print(f"Time: {end_time - start_time}")


Time: 14.544921398162842


In [10]:
def decode_candidates(c):
    candidates = []
    for row in c:
        new_row = []
        for elem in row:
            new_row.append([i + 1 for i, e in enumerate(elem) if e])
        candidates.append(new_row)
    
    return candidates

In [14]:
print(sudoku)

- - - 8 - 1 - - - 
- - - - - - - 4 3 
5 - - - - - - - - 
- - - - 7 - 8 - - 
- - - - - - 1 - - 
- 2 - - 3 - - - - 
6 - - - - - - 7 5 
- - 3 4 - - - - - 
- - - 2 - - 6 - - 



In [15]:
grid_of_candidates = decode_candidates(c)
for row in grid_of_candidates:
    for cell in row:
        if cell:
            print("".join([str(i) for i in cell]), end=" ")
        else:
            print("-1", end=" ")
    print()


23479 34679 24679 -1 24569 -1 2579 2569 2679 
12789 16789 126789 5679 2569 25679 2579 -1 -1 
-1 1346789 1246789 3679 2469 234679 279 12689 126789 
1349 134569 14569 1569 -1 24569 -1 23569 2469 
34789 3456789 456789 569 245689 245689 -1 23569 24679 
14789 -1 1456789 1569 -1 45689 4579 569 4679 
-1 1489 12489 139 189 389 2349 -1 -1 
12789 15789 -1 -1 15689 56789 29 1289 1289 
14789 145789 145789 -1 1589 35789 -1 1389 1489 
